In [16]:
# coding=utf8
import requests, json
import os
import sqlite3
from multiprocessing import Pool
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
import functools
import statistics

import mwparserfromhell
from googleapiclient import discovery, errors

from settings import DATA_FOLDER, API_KEY

DATABASE_PATH = os.path.join(DATA_FOLDER, 'WikiDB.db')
TOXICITY_THRESHOLD = .25


def handle_comment(comment_row):
    print(comment_row)
    article_id, text = comment_row[0], comment_row[1]
    start_tox = time()
    if toxicity_score(text) > TOXICITY_THRESHOLD:
        toxic = True
    else:
        toxic = False
    print(time() - start_tox)
    return article_id, toxic


def article_iterator_wrapper(comment_iterator):
    comments = []
    current_id = 1
    for article_id, comment in comment_iterator:
        if article_id == current_id:
            comments.append(comment)
        else:
            yield current_id, comments
            current_id = article_id
            comments = [comment]

In [2]:
def toxicity_score(comment):
    for i in range(100):
        try:
            if len(comment) > 3000:
                return -1
            service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)
            analyze_request = {
              'comment': { 'text': comment },
              'requestedAttributes': {'TOXICITY': {}}
            }
            try:
                response = service.comments().analyze(body=analyze_request).execute()
            except errors.HttpError:
                return -1
            return response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
        except:
            print("Toxicity API unresponsive ; retrying in " + str(4*i*i) + "s")
            sleep(4*i*i)

In [3]:
conn = sqlite3.connect(DATABASE_PATH)
cur = conn.cursor()

In [ ]:
final_index = json.load(open(os.path.join(DATA_FOLDER, '2110_final_index.json')))
reverse_index = json.load(open(os.path.join(DATA_FOLDER, '2110_reverse_index.json')))
edit_wars = json.load(open(os.path.join(DATA_FOLDER, 'edit_wars.json')))

In [ ]:
article_list = []
article_dict = {}
article_iterator = cur.execute("SELECT * from article ORDER BY id")
for i, article in enumerate(article_iterator):
    article_list.append(article[1])
    article_dict[article[1]] = i

In [ ]:
comment_iterator = cur.execute("SELECT article_id, text from comment ORDER BY article_id")
start_time = time()
total_comments = 0
max_comments = 0
comment_distribution = np.zeros(47628)
comment_distribution_with_titles = []
for i in range(47628):
    comment_distribution_with_titles.append([])
# pool = Pool(1)

for i, (current_id, comments) in enumerate(article_iterator_wrapper(comment_iterator)):
    talk_length = len(comments)
    total_comments += talk_length
    comment_distribution[talk_length - 1] += 1
    comment_distribution_with_titles[talk_length - 1].append(article_list[current_id - 1])
    if talk_length > max_comments:
        max_comments = talk_length
    if i % 1000 == 0:
        print(time()-start_time)
        print(i)


print(max_comments)
print(total_comments / float(i))

In [ ]:
cumulative_distribution = np.cumsum(comment_distribution)

cumulative_distribution[-1] - cumulative_distribution[150]

cumulative_distribution[200] - cumulative_distribution[150]

print(comment_distribution_with_titles[175])

In [6]:
def get_toxicity(row):
    comment_id, text, previous_score = row[0], row[1], row[2]
    if previous_score is not None:
        return None, comment_id
    cleaned_text = mwparserfromhell.parse(text).strip_code()
    toxicity = toxicity_score(cleaned_text)
    return toxicity, comment_id    
    
    
def get_toxicity_for_cleanup(row):
    comment_id, text = row[0], row[1]
    return toxicity_score(mwparserfromhell.parse(text).strip_code()), comment_id
    
    
def fetch_toxicity(articles_to_analyze):
    start_time = time()
    toxicity_dict = {}
    ids_to_analyze = [article_dict.get(article, None) + 1 for article in articles_to_analyze if article_dict.get(article, None) is not None]
    placeholder= '?'
    placeholders= ', '.join(placeholder for item in ids_to_analyze)
    query= 'SELECT id, text, toxicity FROM comment WHERE article_id IN (%s)' % placeholders
    comment_iterator = cur.execute(query, ids_to_analyze)
    comments_to_analyze = comment_iterator.fetchall()
    print('total to analyze : ' + str(len(comments_to_analyze)))
    start_loop = time()
    print(start_loop - start_time)
    
    pool = Pool(3)
    
    for i, (toxicity, comment_id) in enumerate(pool.imap_unordered(get_toxicity, comments_to_analyze)):
        if toxicity is not None:
            toxicity_dict[comment_id] = toxicity
        if i % 1000 == 0:
            print(i)
            print(time() - start_loop)
    print(i)
    print(time() - start_loop)
    
    results = [(toxicity, comment_id) for comment_id, toxicity in toxicity_dict.items()]
    query = "UPDATE comment SET toxicity = ? WHERE id = ?"
    cur.executemany(query, results)
    conn.commit()
    
    
def fetch_toxicity_by_id(final_ids_to_analyze):
    start_time = time()
    toxicity_dict = {}
    
    placeholder= '?'
    placeholders= ', '.join(placeholder for item in final_ids_to_analyze)
    query= 'SELECT id FROM article WHERE talk_length < 200 AND final_id IN (%s)' % placeholders
    comment_iterator = cur.execute(query, final_ids_to_analyze)
    ids_to_analyze = [row[0] for row in comment_iterator]
    
    placeholder= '?'
    placeholders= ', '.join(placeholder for item in ids_to_analyze)
    query= 'SELECT id, text, toxicity FROM comment WHERE parent_id != 0 AND toxicity IS NULL AND article_id IN (%s)' % placeholders
    comment_iterator = cur.execute(query, ids_to_analyze)
    start_loop = time()
    print(start_loop - start_time)
    comments_to_analyze = comment_iterator.fetchall()
    print('total to analyze : ' + str(len(comments_to_analyze)))
    
    pool = Pool(3)
    
    for i, (toxicity, comment_id) in enumerate(pool.imap_unordered(get_toxicity, comments_to_analyze)):
        if toxicity is not None:
            toxicity_dict[comment_id] = toxicity
        if i % 1000 == 0:
            print(i)
            print(time() - start_loop)
    print(i)
    print(time() - start_loop)
    
    results = [(toxicity, comment_id) for comment_id, toxicity in toxicity_dict.items()]
    query = "UPDATE comment SET toxicity = ? WHERE id = ?"
    cur.executemany(query, results)
    conn.commit()
    
    
def toxicity_iterator(comment_iterator):
    comments = []
    current_id = None
    current_talk_length = None
    for article_id, toxicity, talk_length in comment_iterator:
        if article_id == current_id:
            comments.append(toxicity)
        elif current_id is None:
            current_id = article_id
            current_talk_length = talk_length
            comments = [toxicity]
        else:
            yield current_id, comments, current_talk_length
            current_talk_length = talk_length
            current_id = article_id
            comments = [toxicity]
    

def iterate_toxicity_by_id(final_ids_to_analyze):
    start_time = time()
    toxicity_dict = {}
    
    placeholder= '?'
    placeholders= ', '.join(placeholder for item in final_ids_to_analyze)
    query= 'SELECT final_id, toxicity, talk_length FROM comment INNER JOIN article ON comment.article_id = article.id WHERE toxicity IS NOT NULL AND parent_id != 0 AND final_id IN (%s) ORDER BY final_id' % placeholders
    comment_iterator = cur.execute(query, final_ids_to_analyze)
    start_loop = time()
    print(start_loop - start_time)
    
    return toxicity_iterator(comment_iterator)


def clean_toxicity():
    start_time = time()
    query= 'SELECT id, text FROM comment WHERE toxicity = -1'
    comment_iterator = cur.execute(query)
    comments_to_analyze = comment_iterator.fetchall()
    print('total to analyze : ' + str(len(comments_to_analyze)))
    toxicity_dict = {}
    
    problems = 0
    pool = Pool(3)
    
    for i, (toxicity, comment_id) in enumerate(pool.imap_unordered(get_toxicity_for_cleanup, comments_to_analyze)):
        if toxicity != -1:
            problems += 1
            toxicity_dict[comment_id] = toxicity
        if i % 1000 == 0:
            print(i)
            print(str(problems) + ' problems')
            print(time() - start_time)
    print(i)
    print(time() - start_time)
    
    print(str(problems) + ' problems')
    results = [(toxicity, comment_id) for comment_id, toxicity in toxicity_dict.items()]
    query = "UPDATE comment SET toxicity = ? WHERE id = ?"
    cur.executemany(query, results)
    conn.commit()

In [ ]:
query = 'SELECT article_id, toxicity FROM comment WHERE toxicity IS NOT NULL ORDER BY article_id'
comment_iterator = cur.execute(query)
toxic_comments = comment_iterator.fetchall()

for article_id, toxicities in toxicity_iterator(toxic_comments):
    print(article_list[article_id + 1])
    print(len([value for value in toxicities if value > 0.25]))

In [7]:
edit_wars_by_id = {final_index.get(article_title.strip(), None): value for article_title, value in edit_wars.items() if final_index.get(article_title.strip(), None) is not None}

In [8]:
small_wars = [int(item) for item, value in edit_wars_by_id.items() if value == 1]
big_wars = [int(item) for item, value in edit_wars_by_id.items() if value != 1]

In [9]:
fetch_toxicity_by_id(small_wars + big_wars)

9.119564771652222
total to analyze : 25728
0
120.17145609855652
1000
213.73272156715393
2000
307.5810799598694
3000
401.2042136192322
4000
494.2507061958313
5000
587.5135655403137
6000
682.3595886230469
7000
775.1076378822327
8000
867.5104930400848
9000
960.6196150779724
10000
1052.2423701286316
11000
1145.041989326477
12000
1239.984529018402
13000
1331.3330903053284
14000
1423.7848436832428
15000
1515.0919589996338
16000
1609.2695860862732
17000
1701.4609694480896
18000
1793.3942289352417
19000
1886.3059031963348
20000
1976.8551437854767
21000
2068.565089225769
22000
2159.4753239154816
23000
2251.7271423339844
24000
2345.8577468395233
25000
2439.500079870224
25727
2506.8995604515076


In [10]:
active_edit_wars_by_id = []
for i, (final_id, toxicities, talk_length) in enumerate(iterate_toxicity_by_id(small_wars + big_wars)):
    if len(toxicities) > 6 or talk_length > 200 :
        active_edit_wars_by_id.append(final_id)
print(len(active_edit_wars_by_id))

158.51154279708862
5751


In [12]:
query = 'SELECT final_id FROM article WHERE talk_length > 200'
article_iterator = cur.execute(query)
big_articles = set(row[0] for row in article_iterator.fetchall())
print(len(big_articles))

9237


In [5]:
final_dataset = big_articles.union(active_edit_wars_by_id)
print(len(final_dataset))
json.dump(list(final_dataset), open(os.path.join(DATA_FOLDER, '2601_conflicts.json'), 'w'), indent=2)

NameError: name 'big_articles' is not defined

In [ ]:
query = 'SELECT id, text, toxicity FROM comment INNER JOIN article ON comment.article_id = article.id WHERE parent_id != 0 AND talk_length < 200 AND toxicity IS NOT NULL ORDER BY final_id'

In [4]:
big_dataset = json.load(open(os.path.join(DATA_FOLDER, '2601_conflicts.json')))

In [5]:
placeholder= '?'
placeholders= ', '.join(placeholder for item in big_dataset)
query = 'SELECT user_id, final_id from comment INNER JOIN article ON comment.article_id = article.id WHERE final_id IN (%s) ORDER BY final_id' % placeholders
comment_iterator = cur.execute(query, big_dataset)

In [6]:
article_users = {final_id: [] for final_id in big_dataset}
for user_id, final_id in comment_iterator:
    article_users[final_id].append(user_id)
json.dump(article_users, open(os.path.join(DATA_FOLDER, '2601_article_users.json'), 'w'), indent=2)

In [6]:
article_users = json.load(open(os.path.join(DATA_FOLDER, '2601_article_users.json')))

In [7]:
all_users = {user for article_list in article_users.values() for user in article_list }

In [10]:
len(all_users)

531795

In [11]:
sum(len(article_list) for article_list in article_users.values())

6625525

In [8]:
users_articles = {user: [] for user in all_users}
for article, users in article_users.items():
    for user in users:
        users_articles[user].append(article)

In [17]:
sorted([len(set(articles)) for articles in users_articles.values()], reverse=True)

[7829,
 1807,
 1635,
 1151,
 873,
 858,
 812,
 730,
 724,
 664,
 646,
 623,
 605,
 594,
 590,
 588,
 538,
 532,
 525,
 518,
 514,
 512,
 509,
 494,
 487,
 477,
 475,
 459,
 459,
 459,
 459,
 457,
 456,
 450,
 443,
 440,
 440,
 438,
 438,
 433,
 432,
 431,
 428,
 428,
 424,
 422,
 418,
 418,
 410,
 409,
 399,
 396,
 393,
 389,
 388,
 381,
 381,
 376,
 376,
 375,
 373,
 369,
 369,
 368,
 367,
 359,
 353,
 353,
 352,
 344,
 343,
 342,
 342,
 338,
 337,
 335,
 334,
 328,
 324,
 324,
 321,
 320,
 319,
 319,
 318,
 318,
 317,
 316,
 315,
 314,
 314,
 311,
 311,
 311,
 306,
 304,
 303,
 303,
 301,
 301,
 300,
 298,
 295,
 295,
 294,
 294,
 294,
 292,
 290,
 289,
 287,
 287,
 284,
 284,
 284,
 280,
 280,
 280,
 280,
 278,
 276,
 275,
 275,
 274,
 273,
 273,
 273,
 273,
 272,
 272,
 268,
 265,
 264,
 263,
 263,
 263,
 262,
 261,
 261,
 259,
 259,
 258,
 258,
 258,
 254,
 253,
 251,
 250,
 249,
 249,
 248,
 247,
 247,
 245,
 245,
 244,
 243,
 242,
 241,
 241,
 240,
 239,
 238,
 238,
 238,
 238,


In [9]:
bots = [user for user in users_articles if len(set(users_articles[user])) > 1000]

In [11]:
users_articles_no_bots = {user: articles for user, articles in users_articles.items() if user not in bots}

In [17]:
median_articles = statistics.median((len(set(articles)) for articles in users_articles_no_bots.values()))

In [18]:
print(median_articles)

1


In [22]:
placeholder= '?'
placeholders= ', '.join(placeholder for item in big_dataset)
query = 'SELECT user_id, final_id from comment INNER JOIN article ON comment.article_id = article.id WHERE toxicity > 0.25 AND parent_id != 0 AND final_id IN (%s) ORDER BY final_id' % placeholders
comment_iterator = cur.execute(query, big_dataset)

In [23]:
toxic_article_users = {final_id: [] for final_id in big_dataset}
for user_id, final_id in comment_iterator:
    toxic_article_users[final_id].append(user_id)
toxic_users = {user for article_list in toxic_article_users.values() for user in article_list }

11539
29501


In [31]:
toxic_article_users = {k: v for k, v in toxic_article_users.items() if len(v) != 0}

In [32]:
print(len(toxic_users))
print(statistics.median(len(set(article_list)) for article_list in toxic_article_users.values()))

11539
2.0
